# Homework Week 04
Nicola Zomer <br>
10 November, 2022

## Assignment
1) Generate a random a SxS matrix with C non zero entries and 1-C zeros (C is between zero and one). Set the diagonal to -d. The non-zero elements are drawn  from a given distributions. Calculate the eigenvalues. Repeat different realizations and plot all the eigenvalues in the complex plane [Real part (x-axis) and imaginary part (y-axis)]. Compare this result with the expectations from the circular law. 
   
2) Generate the same as above but for mutualistic structure and find how the maximum real eigenvalues scales with S (use S=20,30,40,..100) and compare your numerical finding with the analytical expectations (we did not explicitly have calculated this in class). 
   
3) Analyse the food web using the metrics we have seen in class. The file represent the weighted adjacency matrix of the food web. For the analysis you can binarize (zero and one) the matrix. You find the file in the google drive.
   
4) (optional) Calculate the same as 1) but with for the cascade model or for the nested mutualistic network. 

In [1]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
import pandas as pd

# dash
from jupyter_dash import JupyterDash
from dash import Dash
## from dash import dcc
## from dash import html
from dash.dependencies import Output, State, Input

# plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

%matplotlib inline

# show static images for github
from IPython.display import Image, display
savegit = False

# Point 1 
## The Circular Law
Consider the Generalized Lotka-Volterra Dynamics, describing the interaction of $S$ species through a network:
$$
    \frac{dN_i}{dt} = a_iN_i(t)- N_i(t)\sum_{j=1}^{S} \alpha_{ij}N_j(t) \quad i=1, ..., S
$$
The Jacobian matrix evaluated at the non-trivial equilibrium point of the system is given by:
$$
    J_{ij}=\frac{\partial \dot{N}_i}{\partial N_j}\Bigr\rvert_{\bar{N}^*} = N_i^*\alpha_{ij}
$$

We consider a stationary point $\bar{N}^*=(1, 1, ...., 1)$, which implies that $J_{ij}=\alpha_{ij}$, and we model $\alpha$ as a random matrix, for example assuming that $\alpha_{ij}$ are drawn from a Gaussian distribution, $\alpha_{ij} \sim \mathcal{N}(0, \sigma^2)$. Assuming interactions are i.i.d. random variables and $\alpha_{ii}=0$ for all $i=1, ..., S$, then all eigenvalues of $\alpha/\sqrt{S\sigma^2}$ are uniformly distributed within the unitary disk in the complex plane. This is called _Circular Law_.

Moreover, if we add the effect of self-interactions and we model it as a constant intra-specied competition, $\alpha_{ii}=-d <0$, then the disk containing all the eigenvalues of $\alpha$ is translated on the real axis, that is:
- the disk center is $(-d, 0)$
- the disk radius is $\sqrt{S\sigma^2}$

Notice that in general $\sigma$ is the standard deviation of the random variable $X$ from which the coefficients $\alpha_{ij}$ take value. 

If we further introduce a constant $C$, called _connectance_, such that with probability $C$ the coefficient of $\alpha_{ij}$ are sampled by the selected distribution and with probability $0$ are set to $0$, than the radius of the disk becomes:
$$
    r= \sqrt{SC\sigma^2}
$$

Pay attention that the Circular Law is verified only if $\alpha$ satisfies 2 conditions:
1. $\mathbb{E}[\alpha_{ij}]=0$, so the interaction matrix has zero mean
2. $\mathbb{E}[\alpha_{ij}, \alpha{ji}]=0$

This conditions depends on the chosen distribution and on how the matrix is built. Notice that they are reached by taking $\mathcal{D} =\mathcal{N}(0, \sigma^2)$ and following the next construction steps. 


## Construction of the Random Matrices
Notation: in this context we call $M_{ij}$ the random matrix, and $\mathcal{D}$ the distribution we have selected (for example $\mathcal{D} =\mathcal{N}(0, \sigma^2)$). 

**Off-diagonal elements** <br>
For each off-diagonal term of the matrix we draw a random value $p$ from a uniform distribution $\mathcal{U}[0, 1]$. If $p\leq C$, we set $M_{ij}$ by sampling it from $\mathcal{D}$, otherwise we impose $M_{ij}=0$

**Diagonal elements** <br>
All diagonal elements, $M_{ii}$, are set to $−d$. 

These matrices, for large S, generate a precise mixture of interaction types, with predator-prey interactions being represented twice as frequently as mutualistic or competitive ones.


In [2]:
# random matrix generation
def randommatrix(size, diag, connectance, D=lambda: np.random.normal(0, 1), mutualistic=False):
    '''
    INPUT:
    - size        -> size of the square matrix (i.e. number of species) 
    - diag        -> value of d (>0)
    - connectance -> value of C (0<C<1)
    - D           -> the selected distribution, by default a gaussian distribution
    - mutualistic -> if True, consider only mutualistic interactions
    
    OUTPUT:
    - M -> the random matrix
    
    '''
    
    assert diag>0, 'diag  must be >0'
    assert 0<connectance<1, 'connectance must be between 0 and 1'
    
    # initialize matrix
    M = np.zeros((size, size))
    
    # 1. off-diagonal elements
    for i in range(size):
        for j in range(i, size):
            p = np.random.uniform()
            
            if p<=connectance:
                if not mutualistic:
                    M[i, j] = D()
                    M[j, i] = D()
                else:
                    M[i, j] = np.abs(D())
                    M[j, i] = np.abs(D())
    
    # 2. diagonal elements
    np.fill_diagonal(M, -diag)

    return M


# circle extremes
def my_circle(center, radius):
    '''
    INPUT:
    - center -> [ , ], center of the disk
    - radius -> float, radius of the disk
    
    OUTPUT: 
    - x0 -> min(x)
    - y0 -> min(y)
    - x1 -> max(x)
    - y1 -> max(y)
    
    '''
    
    x0_ = center[0]-radius
    y0_ = center[1]-radius
    x1_ = center[0]+radius
    y1_ = center[1]+radius

    return x0_, y0_, x1_, y1_


In [3]:
np.random.seed(1245)

# parameters
C = 0.85
d = 1
S = 100
sigma = 1

# center and radius of the circle
center = [-d, 0]
radius = m.sqrt(S*C*sigma**2)

print('The expected center of the disk is: (%.2f, %i)' %(center[0], center[1]))
print('The expected radius of the disk is: r = %.2f' %(radius))


# build N random matrices
N = 10
appended_data = []

for i in range(N):
    df_eig = pd.DataFrame(columns=['re', 'im', 'run'])
    
    M = randommatrix(S, d, C)
    eig_M, _ = np.linalg.eig(M)
    df_eig['re'] = np.real(eig_M)
    df_eig['im'] = np.imag(eig_M)
    df_eig['run'] = [str(i+1)]*len(eig_M)
    
    appended_data.append(df_eig)
    
appended_data = pd.concat(appended_data)

# circle data
x0, y0, x1, y1 = my_circle(center, radius)

# plot the eigenvalues
fig = px.scatter(appended_data, x="re", y="im", color="run")

fig.update_layout(
    title="The Circular Law <br><sup>Eigenvalues in the complex plane</sup>",
    xaxis_title="Re(λ)",
    yaxis_title="Im(λ)",
    legend_title="Run",
    width=600,
    height=600,
)
fig.update_xaxes(range=[-12, 12])
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
    range=[-12, 12]
  )

fig.add_shape(type="circle",
    xref="x", yref="y",
    x0=x0, y0=y0, x1=x1, y1=y1,
    line_color="black",
    line_width = 2.5
)
# fig.add_annotation(
#     x= center[0]+radius,
#     y= center[1],
#     xref="x", yref="y",
#     text="",
#     showarrow=True,
#     axref = "x", ayref='y',
#     ax= center[0],
#     ay= center[1],
#     arrowhead = 3,
#     arrowwidth= 2, 
#     arrowcolor='black')

if savegit: 
    path = "images/hw04_circlaw.png"
    fig.write_image(path)
    display(Image(path))
    
else:
    fig.show()

The expected center of the disk is: (-1.00, 0)
The expected radius of the disk is: r = 9.22


# Point 2
To  generate the same as above for mutualistic structure simply impose that with probability $C$:
$$
    \{\alpha_{ij}, \alpha{ji}\} = \{|x_1|, |x_2|\}
$$ 
where $x_1$ and $x_2$ are sampled from the distribution $\mathcal{D}$.

## Construction of the Random Matrices in the Mutualistic case
**Off-diagonal elements** <br>
For each pair of off-diagonal terms $\{M{ij}, M{ji}\}$ of the matrix we draw a random value $p$ from a uniform distribution $\mathcal{U}[0, 1]$. If $p\leq C$, we set $M_{ij}$ and $M_{ji}$ by sampling them independently from $|\mathcal{D}|$ (which for example can be the half-normal distribution), otherwise we impose $M_{ij}=M_{ji}=0$

**Diagonal elements** <br>
All diagonal elements, $M_{ii}$, are set to $−d$. 


## Ellipse Law and non-zero mean case
First, notice that in this case the correlation coefficient $\rho$, defined as:
$$
    \rho := \frac{\mathbb{E}[M_{ij}, M_{ji}]-\mathbb{E}^2[M_ij]}{Var(M_{ij})}
$$
is different from zero. This implies a modification of the Circular Law, which is called Elliptical Law. According to this law, the eigenvalues of $M$ are uniformly distributed within an ellipse in the complex plane, whose semi-axes are:
$$
\begin{align}
    a &= \sqrt{SV}(1+\rho) \\
    b &= \sqrt{SV}(1-\rho)
\end{align}
$$
where $V=Var(M_{ij})$. 

Moreover, in this case the interaction matrix has non-zero mean. This leads to an eigenvalue, which in general can be located outside of the ellipse and seen as a sort of outlier, such that:
$$
    Re(\lambda) = -d+(S-1)\mathbb{E}[M_ij]
$$ 
Not only this, but also the center of the ellipse is moved in $(-d-\mathbb{E}[M_{ij}], 0)$.



In [4]:
# rho numerical estimate
def correlation_numerical(M):
    '''
    INPUT: matrix M
    OUTPUT: expected correlation rho between Mij and Mji
    
    '''
    
    E_aij = M.mean()
    E_aij_aji = np.mean((M*M.T)[np.triu_indices(M.shape[0])])
    Var_aij = M.var()
    
    return (E_aij_aji - E_aij**2)/Var_aij


# outlier numerical estimate
def outlier_numerical(M, species):
    '''
    INPUT:
    - species  -> float, number of species
    - M        -> interaction matrix M
    
    OUTPUT: Re(lambda)
    
    '''
    
    return -d+(species-1)*M.mean()


# rho mutualistic
def correlation_mutualistic(M):
    pass


# outlier mutualistic
def outlier_mutualistic(M):
    pass


# ellipse extremes
def my_ellipse(center, species, variance, rho):
    '''
    INPUT:
    - center   -> [ , ], center of the ellipse
    - species  -> float, number of species
    - variance -> variance of the interaction matrix
    - rho      -> correlation coefficient
    
    OUTPUT: 
    - x0 -> min(x)
    - y0 -> min(y)
    - x1 -> max(x)
    - y1 -> max(y)
    
    '''
        
    radius = np.sqrt(species*variance)
    a = radius*(1+rho)
    b = radius*(1-rho)
    
    x0_ = center[0]-a
    y0_ = center[1]-b
    x1_ = center[0]+a
    y1_ = center[1]+b

    return x0_, y0_, x1_, y1_

## Numerical approach
Here all results are achived numerically, not taking into account the distribution used. At each iteration
- $\mathbb{E}[M_{ij}]$
- $\mathbb{E}[M_{ij}, M_{ji}]$
- $Var(M_{ij})$

are estimated numerically from the costructed matrix. Their final estimated value is obtained by averaging over the iterations. 

In [5]:
np.random.seed(1245)

# build N random matrices, with mutualistic structure
N = 10
appended_data   = []
rho_values      = []
mean_values     = []
variance_values = []
outlier_values  = []

for i in range(N):
    df_eig = pd.DataFrame(columns=['re', 'im', 'run'])
    
    M = randommatrix(S, d, C, mutualistic=True)
    eig_M, _ = np.linalg.eig(M)
    df_eig['re'] = np.real(eig_M)
    df_eig['im'] = np.imag(eig_M)
    df_eig['run'] = [str(i+1)]*len(eig_M)
    
    appended_data.append(df_eig)
    rho_values.append(correlation_numerical(M))
    variance_values.append(np.var(M))
    mean_values.append(np.mean(M))
    outlier_values.append(outlier_numerical(M, S))
    
appended_data = pd.concat(appended_data)
rho = np.mean(rho_values)
variance_M = np.mean(variance_values)
expected_M = np.mean(mean_values)

out_lambda = np.mean(outlier_values)

# ellipse data
center = [-d-expected_M, 0]
print('The expected center of the ellipse is: (%.2f, %i)' %(center[0], center[1]))

x0, y0, x1, y1 = my_ellipse(center, S, variance_M, rho)

# estimate outlier
print('Outlier eigenvalue: %.2f' %out_lambda)


The expected center of the ellipse is: (-1.66, 0)
Outlier eigenvalue: 64.22


In [6]:
# plot data
df1 = appended_data[appended_data['re']<20]
df2 = appended_data[appended_data['re']>20]

# organize datafames with different x-axes in a dict
dfs = {'df1': df1,
       'df2': df2
}

# subplot setup
colors = px.colors.qualitative.Plotly
fig = make_subplots(rows=1, cols=len(dfs.keys()), horizontal_spacing = 0.02, subplot_titles=("Ellipse", "Oulier"))
fig.update_layout(title = "Broken / discontinued / gapped x-axis")

# Assign columns from dataframes in dict to the correct subplot
for i, dfd in enumerate(dfs, start =1):
    for j, run in enumerate(dfs[dfd]['run'].unique()):
        df_ = dfs[dfd]
        df_ = df_[df_['run']==run]
        fig.add_trace(go.Scatter(x=df_['re'],
                        y=df_['im'],
                        mode = 'markers',
                        name=run,
                        marker_color=colors[j],
                        legendgroup = run,
                        showlegend = True if i == 1 else False,
                        ), row=1, col=i)

# ellipse
fig.add_shape(type="circle",
    xref="x", yref="y",
    x0=x0, y0=y0, x1=x1, y1=y1,
    line_color="black",
    line_width = 2.5
)

# predicted outlier 
fig.add_trace(go.Scatter(x=[out_lambda], y=[0], 
                        mode = 'markers',
                        marker_color = 'red',
                        marker_symbol = 'star',
                        showlegend=False,
                        marker_size = 15), row=1, col=2)

fig.add_annotation(x=out_lambda, y=0,
            text="Predicted Outlier",
            showarrow=True,
            arrowhead=2,
            arrowsize=1.5,
            arrowwidth=1,
            row=1, col=2)

# make the plot nice
fig.update_yaxes(range=[-12, 12], )
fig.update_yaxes(showticklabels=False, row=1, col=2)

fig.update_xaxes(range=[-12, 12], title_text='Re(λ)', row=1, col=1)
fig.update_xaxes(range=[62, 70], title_text='Re(λ)', row=1, col=2)

# a little aesthetic adjustment
fig.add_shape(type="line",
    x0=0.50, y0=0.5-0.015, x1=0.508, y1=0.5+0.015,
    line=dict(color="grey",width=1),
    xref = 'paper',
    yref = 'paper'
)

fig.add_shape(type="line",
    x0=0.492, y0=0.5-0.015, x1=0.5, y1=0.5+0.015,
    line=dict(color="grey",width=1),
    xref = 'paper',
    yref = 'paper'
)

# layout
fig.update_layout(
    height=600, width=1000, 
    title_text="The Elliptical Law with non-zero mean of M <br><sup>Eigenvalues in the complex plane, numerical estimate</sup>", 
    yaxis_title="Im(λ)",
    legend_title="Run")

if savegit: 
    path = "images/hw04_ellipticallaw_numerical.png"
    fig.write_image(path)
    display(Image(path))
    
else:
    fig.show()
    

## Analytical expectations
The above quantities can be computed analytically, in particular considering the case $\mathcal{D}=\mathcal{N}(0, \sigma^2)$. In this case the values of $M_{ij}$ and $M_{ji}$ are drawn independently from a half-normal distribution. 

The expected value of $M_{ij}$ can be easily obtained from the expected value of the half-normal distribution, indeed:
$$
    \mathbb{E}[M_{ij}] = C \cdot\mathbb{E}[X]\bigr|_{X\sim |\mathcal{N}(0, \sigma^2)|} = C \cdot \frac{2\sigma}{\sqrt{2\pi}}
$$

Similarly, it is possible to compute $\mathbb{E}[M_{ij}, M_{ji}]$:
$$
    \mathbb{E}[M_{ij}, M_{ji}] = C \cdot\mathbb{E}^2[X]\bigr|_{X\sim |\mathcal{N}(0, \sigma^2)|} = C \cdot \frac{2\sigma^2}{\pi}
$$

Finally, we can get also the variance of $M_{ij}$:
$$
    Var(M_{ij}) = \mathbb{E}[M^2_{ij}] - \mathbb{E}^2[M_{ij}] = C\left[\sigma^2-C\cdot\frac{2\sigma^2}{\pi}\right] = C\sigma^2\left(1-\frac{2C}{\pi}\right)
$$

Having all these quantities, we can derive the correlation $\rho$ and the semi-axes of the ellipse:
$$
\begin{align}
    &\rho = \frac{\mathbb{E}[M_{ij}, M_{ji}] - \mathbb{E}^2[M_{ij}]}{Var(M_{ij})} = \frac{C \cdot\frac{2\sigma^2}{\pi} - C^2 \cdot\frac{2\sigma^2}{\pi}}{C\cdot\sigma^2 \left(1-\frac{2C}{\pi} \right)} = \frac{1 - C}{\frac{\pi}{2}-C}\\
    &a = \sqrt{SV}(1 +\rho) = \sqrt{SC\sigma^2 \left( 1 - \frac{2C}{\pi} \right)} \cdot\left(1 + \frac{1-C}{\frac{\pi}{2}-C}\right) \\
    &b = \sqrt{SV}(1 -\rho) = \sqrt{SC\sigma^2 \left( 1 - \frac{2C}{\pi} \right)} \cdot\left(1 - \frac{1-C}{\frac{\pi}{2}-C}\right) 
\end{align}
$$

Finally, the eigenvalue that comes from the fact that the interaction matrix has non-zero mean can be computed as:
$$
    Re(\lambda) = -d+(S-1)\mathbb{E}[M_ij] = -d+C(S-1)\frac{2\sigma}{\sqrt{2\pi}}
$$  




   

In [7]:
np.random.seed(1245) # same as above

# build N random matrices, with mutualistic structure
N = 10
appended_data   = []

for i in range(N):
    df_eig = pd.DataFrame(columns=['re', 'im', 'run'])
    
    M = randommatrix(S, d, C, mutualistic=True)
    eig_M, _ = np.linalg.eig(M)
    df_eig['re'] = np.real(eig_M)
    df_eig['im'] = np.imag(eig_M)
    df_eig['run'] = [str(i+1)]*len(eig_M)
    
    appended_data.append(df_eig)
    
appended_data = pd.concat(appended_data)

# variance and rho
V = C*sigma**2*(1-(2*C)/np.pi)
rho =(1-C)/((np.pi/2)-C)

# ellipse data
center = [-d-C*(2*sigma)/np.sqrt(2*np.pi), 0]
print('The expected center of the ellipse is: (%.2f, %i)' %(center[0], center[1]))

x0, y0, x1, y1 = my_ellipse(center, S, V, rho)

# estimate outlier
out_lambda =  -d + (S-1)*C*(2*sigma)/(np.sqrt(2*np.pi))
print('Outlier eigenvalue: %.2f' %out_lambda)


The expected center of the ellipse is: (-1.68, 0)
Outlier eigenvalue: 66.14


In [8]:
# plot data
df1 = appended_data[appended_data['re']<20]
df2 = appended_data[appended_data['re']>20]

# organize datafames with different x-axes in a dict
dfs = {'df1': df1,
       'df2': df2
}

# subplot setup
colors = px.colors.qualitative.Plotly
fig = make_subplots(rows=1, cols=len(dfs.keys()), horizontal_spacing = 0.02, subplot_titles=("Ellipse", "Oulier"))
fig.update_layout(title = "Broken / discontinued / gapped x-axis")

# Assign columns from dataframes in dict to the correct subplot
for i, dfd in enumerate(dfs, start =1):
    for j, run in enumerate(dfs[dfd]['run'].unique()):
        df_ = dfs[dfd]
        df_ = df_[df_['run']==run]
        fig.add_trace(go.Scatter(x=df_['re'],
                        y=df_['im'],
                        mode = 'markers',
                        name=run,
                        marker_color=colors[j],
                        legendgroup = run,
                        showlegend = True if i == 1 else False,
                        ), row=1, col=i)

# ellipse
fig.add_shape(type="circle",
    xref="x", yref="y",
    x0=x0, y0=y0, x1=x1, y1=y1,
    line_color="black",
    line_width = 2.5
)

# predicted outlier 
fig.add_trace(go.Scatter(x=[out_lambda], y=[0], 
                        mode = 'markers',
                        marker_color = 'red',
                        marker_symbol = 'star',
                        showlegend=False,
                        marker_size = 15), row=1, col=2)

fig.add_annotation(x=out_lambda, y=0,
            text="Predicted Outlier",
            showarrow=True,
            arrowhead=2,
            arrowsize=1.5,
            arrowwidth=1,
            row=1, col=2)

# make the plot nice
fig.update_yaxes(range=[-12, 12], )
fig.update_yaxes(showticklabels=False, row=1, col=2)

fig.update_xaxes(range=[-12, 12], title_text='Re(λ)', row=1, col=1)
fig.update_xaxes(range=[62, 70], title_text='Re(λ)', row=1, col=2)

# a little aesthetic adjustment
fig.add_shape(type="line",
    x0=0.50, y0=0.5-0.015, x1=0.508, y1=0.5+0.015,
    line=dict(color="grey",width=1),
    xref = 'paper',
    yref = 'paper'
)

fig.add_shape(type="line",
    x0=0.492, y0=0.5-0.015, x1=0.5, y1=0.5+0.015,
    line=dict(color="grey",width=1),
    xref = 'paper',
    yref = 'paper'
)

# layout
fig.update_layout(
    height=600, width=1000, 
    title_text="The Elliptical Law with non-zero mean of M <br><sup>Eigenvalues in the complex plane, analytical calculation</sup>", 
    yaxis_title="Im(λ)",
    legend_title="Run")

if savegit: 
    path = "images/hw04_ellipticallaw_analytical.png"
    fig.write_image(path)
    display(Image(path))
    
else:
    fig.show()
    

## Scaling of the Maximum Eighenvalue

In [9]:
np.random.seed(1245)

# setting
S_vec = np.arange(20, 110, 10).astype(int)  # number of species
N = 10                                      # number of iterations for each # species (take the average value)

df_max_re = pd.DataFrame(columns=['Maxeig', 'S'])

# build the random matrices and find the max real eigenvalue
for s in S_vec:
    max_eig = []
    for i in range(N):
        M = randommatrix(s, d, C, mutualistic=True)
        eig_M, _ = np.linalg.eig(M)
        
        max_eig.append(np.max(np.real(eig_M)))
        
    
    df_max_re = df_max_re.append({'Maxeig': np.average(max_eig), 'S':str(s)}, ignore_index=True)
        
display(df_max_re.round(2))

# plot
fig = px.scatter(df_max_re, x="S", y="Maxeig")

fig.update_layout(
    title="Scaling of the Max Real Eigenvalue with S",
    xaxis_title="S",
    yaxis_title="Max Re(λ)",
    width=900,
    height=400
)
fig.update_traces(
    marker = dict(size=8,
                  line=dict(width=1, color='Black')
    )
)

if savegit: 
    path = "images/hw04_scaling_maxeig.png"
    fig.write_image(path)
    display(Image(path))
    
else:
    fig.show()

,Maxeig,S
0,12.30,20
1,18.73,30
2,25.60,40
3,32.31,50
4,38.84,60
5,45.94,70
6,52.71,80
7,59.38,90
8,66.43,100


# Additional 1: other structures
- Predator - Prey Structure
- Competitive + Mutualistic Structure

In [10]:
# random matrix generation, competitive + mutualistic or predator-prey
def randommatrix_cm_pp(size, diag, connectance, D=lambda: np.random.normal(0, 1), competitivemutualistic=True):
    '''
    INPUT:
    - size                   -> size of the square matrix (i.e. number of species) 
    - diag                   -> value of d (>0)
    - connectance            -> value of C (0<C<1)
    - D                      -> the selected distribution, by default a gaussian distribution
    - competitivemutualistic -> if true, competitive + mutualistic, else predator-prey interaction
    
    OUTPUT:
    - M -> the random matrix
    
    '''
    
    assert diag>0, 'diag  must be >0'
    assert 0<connectance<1, 'connectance must be between 0 and 1'
    
    # initialize matrix
    M = np.zeros((size, size))
    
    # 1. off-diagonal elements
    for i in range(size):
        for j in range(i+1, size):
            p = np.random.uniform()
                
            if p<=connectance/2:
                if competitivemutualistic: 
                    # mutualistic pair of species
                    M[i, j] = np.abs(D())
                    M[j, i] = np.abs(D())
                else:
                    # predator-prey interaction (ij>0, ji<0)
                    M[i, j] = np.abs(D())
                    M[j, i] = -np.abs(D())
                
            elif connectance/2<p<=connectance:
                if competitivemutualistic:
                    # competitive pair of species
                    M[i, j] = -np.abs(D())
                    M[j, i] = -np.abs(D())
                else:
                    # predator-prey interaction (ij<0, ji>0)
                    M[i, j] = -np.abs(D())
                    M[j, i] = np.abs(D())
    
    # 2. diagonal elements
    np.fill_diagonal(M, -diag)

    return M


# rho knowing the distribution
def correlation_zeromean(M, connectance, sigma):
    '''
    INPUT: matrix M, connectance, standard deviation sigma (mean is taken as zero)
    OUTPUT: expected correlation rho between Mij and Mji
    
    '''
    E_aij_aji = np.mean((M*M.T)[np.triu_indices(M.shape[0])])
    
    return (E_aij_aji)/(connectance*sigma**2)

In [11]:
np.random.seed(1245)

# parameters
C = 0.85
d = 1
S = 100
sigma = 1

# center and radius of the circle
center = [-d, 0]
radius = m.sqrt(S*C*sigma**2)

# initialize lists
N = 10
appended_data_cm = []
appended_data_pp = []

# build N random matrices with competitive + mutualistic structure
rho_values = []

for i in range(N):
    df_eig = pd.DataFrame(columns=['re', 'im', 'run'])
    
    M = randommatrix_cm_pp(S, d, C, competitivemutualistic=True)
    eig_M, _ = np.linalg.eig(M)
    df_eig['re'] = np.real(eig_M)
    df_eig['im'] = np.imag(eig_M)
    df_eig['run'] = [str(i+1)]*len(eig_M)
    
    appended_data_cm.append(df_eig)
    rho_values.append(correlation_zeromean(M, C, sigma))
    
appended_data_cm = pd.concat(appended_data_cm)
rho_cm = np.mean(rho_values)

# build N random matrices with predator-prey structure
rho_values = []

for i in range(N):
    df_eig = pd.DataFrame(columns=['re', 'im', 'run'])
    
    M = randommatrix_cm_pp(S, d, C, competitivemutualistic=False)
    eig_M, _ = np.linalg.eig(M)
    df_eig['re'] = np.real(eig_M)
    df_eig['im'] = np.imag(eig_M)
    df_eig['run'] = [str(i+1)]*len(eig_M)
    
    appended_data_pp.append(df_eig)
    rho_values.append(correlation_zeromean(M, C, sigma))
    
appended_data_pp = pd.concat(appended_data_pp)
rho_pp = np.mean(rho_values)

# ellipses data
x0_cm, y0_cm, x1_cm, y1_cm = my_ellipse(center, radius, rho_cm)
x0_pp, y0_pp, x1_pp, y1_pp = my_ellipse(center, radius, rho_pp)

# plot cm eigenvalues
fig = px.scatter(appended_data_cm, x="re", y="im", color="run")

fig.update_layout(
    title="The Circular Law, competitive and mutualistic interactions<br><sup>Eigenvalues in the complex plane</sup>",
    xaxis_title="Re(λ)",
    yaxis_title="Im(λ)",
    legend_title="Run",
    width=600,
    height=600,
)
fig.update_xaxes(range=[-18, 18])
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
    range=[-18, 18]
  )

fig.add_shape(type="circle",
    xref="x", yref="y",
    x0=x0_cm, y0=y0_cm, x1=x1_cm, y1=y1_cm,
    line_color="black",
    line_width = 2.5
)

if savegit: 
    path = "images/hw04_compmutual.png"
    fig.write_image(path)
    display(Image(path))
    
else:
    fig.show()

# plot pp eigenvalues
fig = px.scatter(appended_data_pp, x="re", y="im", color="run")

fig.update_layout(
    title="The Circular Law, predator-prey interactions<br><sup>Eigenvalues in the complex plane</sup>",
    xaxis_title="Re(λ)",
    yaxis_title="Im(λ)",
    legend_title="Run",
    width=600,
    height=600,
)
fig.update_xaxes(range=[-18, 18])
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
    range=[-18, 18]
  )

fig.add_shape(type="circle",
    xref="x", yref="y",
    x0=x0_pp, y0=y0_pp, x1=x1_pp, y1=y1_pp,
    line_color="black",
    line_width = 2.5
)

if savegit: 
    path = "images/hw04_predatorprey.png"
    fig.write_image(path)
    display(Image(path))
    
else:
    fig.show()
    

TypeError: my_ellipse() missing 1 required positional argument: 'rho'

# Additional 2: Dashboard

Idee per la dashboard: 
- S fisso
- d, C imposti dall'utente, con dei vincoli (d>0, C<1)
- sigma imposto se seleziono non uniforme
- possibilità di selezionare tra diverse distribuzioni (uniforme, normale, exponential)

# Point 3

# Point 4